# Time Series의 Feature Engineering

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

DataFrame 객체에 'date' 열을 추가하고, 그 안에 datetime 객체들을 넣어 2021년 12월 1일부터 12월 5일까지의 날짜를 생성합니다.
DataFrame 객체에 'birth' 열을 추가하고, 해당 날짜에 대한 출생 수를 기록합니다.

In [2]:
df = pd.DataFrame()
df['date'] = [datetime(2021, 12, 1), datetime(2021, 12, 2), 
                  datetime(2021, 12, 3), datetime(2021, 12, 4), datetime(2021, 12, 5)]
df['birth'] = [35, 32, 30, 31, 44]
df

,date,birth
0,2021-12-01,35
1,2021-12-02,32
2,2021-12-03,30
3,2021-12-04,31
4,2021-12-05,44


## date time feature  

주어진 코드는 데이터 프레임에 날짜 관련 여러 특성을 추가합니다. 다음과 같은 작업을 수행합니다:

- 원본 데이터 프레임(df)을 복사하여 새로운 데이터 프레임(features)을 생성합니다.
- 'date' 열의 datetime 객체로부터 연도 정보를 추출하여 'year' 열을 생성합니다.
- 'date' 열의 datetime 객체로부터 월 정보를 추출하여 'month' 열을 생성합니다.
- 'date' 열의 datetime 객체로부터 일 정보를 추출하여 'day' 열을 생성합니다.

In [3]:
features = df.copy()
features['year']    = df['date'].dt.year
features['month'] = df['date'].dt.month
features['day']     = df['date'].dt.day
features

,date,birth,year,month,day
0,2021-12-01,35,2021,12,1
1,2021-12-02,32,2021,12,2
2,2021-12-03,30,2021,12,3
3,2021-12-04,31,2021,12,4
4,2021-12-05,44,2021,12,5


## Lag features (지연 특성)

주어진 코드는 데이터 프레임에 lag features를 추가합니다. 다음과 같은 작업을 수행합니다:

- 'birth' 열의 데이터를 1칸 앞으로 이동시킨 후, 'lag1' 열에 할당합니다.
- 'birth' 열의 데이터를 3칸 앞으로 이동시킨 후, 'lag2' 열에 할당합니다.

In [4]:
features['lag1'] = df['birth'].shift(1)
features['lag2'] = df['birth'].shift(3)
features

,date,birth,year,month,day,lag1,lag2
0,2021-12-01,35,2021,12,1,NaN,NaN
1,2021-12-02,32,2021,12,2,35.0,NaN
2,2021-12-03,30,2021,12,3,32.0,NaN
3,2021-12-04,31,2021,12,4,30.0,35.0
4,2021-12-05,44,2021,12,5,31.0,32.0


## Window features

### Rolling Window vs. Expanding Window (이동 창과 확장 창)  
이동 창과 확장 창은 데이터를 다루는 방식에 차이가 있습니다.  


100일 동안의 데이터를 예로 들면:

이동 창에서는 창의 크기가 계속 같습니다. 예를 들어 창 크기를 10일로 설정하면, 처음 예측은 1일부터 10일까지의 데이터를 가지고 11일차의 데이터를 예측합니다. 그 다음 예측은 2일부터 11일까지의 데이터를 사용하여 12일차를 예측합니다. 이렇게 창은 항상 10일치 데이터를 유지하며, 시간이 지남에 따라 이동합니다.

반면에 확장 창에서는 창의 크기가 점차 커집니다. 첫 번째 예측은 처음 10일의 데이터를 사용합니다. 그리고 두 번째 예측에서는 10일치 데이터에 하루가 추가되어 총 11일치 데이터를 사용합니다. 이런 식으로 예측할 때마다 하루씩 데이터가 더해져 창의 크기가 점점 확장됩니다.

- 'birth' 열의 데이터에 대해 크기가 2인 이동 평균(rolling mean)을 계산하고, 'Roll_mean' 열에 할당합니다.
- 'birth' 열의 데이터에 대해 크기가 3인 이동 최대값(rolling max)을 계산하고, 'Roll_max' 열에 할당합니다.

In [6]:
features['Roll_mean'] = df['birth'].rolling(window=2).mean()
features['Roll_max']  = df['birth'].rolling(window=3).max()
features

,date,birth,year,month,day,lag1,lag2,Roll_mean,Roll_max
0,2021-12-01,35,2021,12,1,NaN,NaN,NaN,NaN
1,2021-12-02,32,2021,12,2,35.0,NaN,33.5,NaN
2,2021-12-03,30,2021,12,3,32.0,NaN,31.0,35.0
3,2021-12-04,31,2021,12,4,30.0,35.0,30.5,32.0
4,2021-12-05,44,2021,12,5,31.0,32.0,37.5,44.0


'birth' 열의 데이터에 대해 expanding window를 생성하는 방법을 보여줍니다. min_periods 매개변수를 사용하면, 윈도우에서 집계를 시작하는 데 필요한 최소한의 이전 날짜를 설정할 수 있습니다

In [7]:
df['birth'].expanding()       #min_period : aggregate 하는데 필요한 최소한의 이전 date

Expanding [min_periods=1,axis=0,method=single]

'birth' 열의 데이터에 대해 expanding window의 최대값을 계산하고, 이를 features 데이터 프레임에 'Expand_max' 열로 추가합니다. 

In [8]:
features['Expand_max'] = df['birth'].expanding().max()  #current date까지의 최대값
features

,date,birth,year,month,day,lag1,lag2,Roll_mean,Roll_max,Expand_max
0,2021-12-01,35,2021,12,1,NaN,NaN,NaN,NaN,35.0
1,2021-12-02,32,2021,12,2,35.0,NaN,33.5,NaN,35.0
2,2021-12-03,30,2021,12,3,32.0,NaN,31.0,35.0,35.0
3,2021-12-04,31,2021,12,4,30.0,35.0,30.5,32.0,35.0
4,2021-12-05,44,2021,12,5,31.0,32.0,37.5,44.0,44.0


## Downsampling and Upsampling

시계열 데이터를 다룰 때, 우리가 수집한 데이터는 종종 불규칙한 간격으로 기록됩니다. 이런 데이터를 분석하기 위해서는 데이터의 간격을 균일하게 조정하는 과정이 필요합니다. 이를 위해 사용하는 두 가지 기법이 '업샘플링'과 '다운샘플링'입니다.

1. 다운샘플링 (데이터 빈도 줄이기):
   - 데이터가 너무 자주 기록되어 실제로 필요한 것보다 많은 경우, 예를 들어 매 초마다 기록된 데이터를 매 분마다로 줄이는 것입니다.
   - 특정 기간의 트렌드를 더 잘 보기 위해 데이터 포인트의 수를 줄이고 싶을 때 사용합니다.
   - 다른 데이터 세트와 일치시키기 위해, 예를 들어 분 단위 데이터를 시간 단위로 변경하는 것도 포함됩니다.

2. 업샘플링 (더 자세한 데이터 얻기):
   - 기존 데이터 사이에 새로운 데이터 포인트를 추가하여 시간 간격을 더 세밀하게 만들고 싶을 때 사용합니다.
   - 예를 들어, 5분 간격의 데이터를 1분 간격의 데이터로 만들어 더 많은 정보를 얻을 수 있습니다.
   - 불규칙한 시계열을 균일한 간격으로 만들거나, 여러 데이터 세트를 같은 빈도로 맞춰야 할 때 유용합니다.

간단히 말해서, 다운샘플링은 데이터를 '희석'시키는 과정이고, 업샘플링은 데이터를 '집중'시키는 과정입니다. 이를 통해 분석하고자 하는 주기에 맞게 데이터의 간격을 조정할 수 있습니다.

샘플링을 위한 테스트 데이터를 생성합니다.  
pd.date_range 함수를 사용하여 2015년 2월 24일부터 시작하는 시간 인덱스를 생성하고, 분단위로 10개의 기간을 생성합니다(freq='T'는 minutely frequency를 의미합니다).  
생성된 시간 인덱스를 기반으로 한 데이터 프레임을 생성하고, 임의의 난수 값으로 구성된 'Val' 열을 추가합니다.

In [9]:
np.random.seed(0)

# '2015-02-24'부터 시작하여 분(minute) 단위로 10개의 연속된 시간 데이터를 생성합니다. ('T'는 minutely frequency를 의미합니다.)
rng = pd.date_range('2015-02-24', periods=10, freq='T') 

# 생성된 시간 데이터를 인덱스로 하고, 정규 분포에서 추출된 난수를 값으로 하는 DataFrame을 생성합니다.
df = pd.DataFrame({'Val' : np.random.randn(len(rng))}, index=rng) 
df

,Val
2015-02-24 00:00:00,1.764052
2015-02-24 00:01:00,0.400157
2015-02-24 00:02:00,0.978738
2015-02-24 00:03:00,2.240893
2015-02-24 00:04:00,1.867558
2015-02-24 00:05:00,-0.977278
2015-02-24 00:06:00,0.950088
2015-02-24 00:07:00,-0.151357
2015-02-24 00:08:00,-0.103219
2015-02-24 00:09:00,0.410599


### Downsampling

pd.date_range(start, end, periods, freq)  
[frequency alias](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases) 참조

pd.date_range 함수를 사용하여 2000년 1월 1일부터 시작하는 시간 인덱스를 생성하고, 분단위로 9개의 기간을 생성합니다(freq='T'는 minutely frequency를 의미합니다).  
생성된 시간 인덱스와 범위(range)에서 0부터 8까지의 정수로 구성된 pandas 시리즈(series)를 생성합니다.

In [10]:
# '2000-01-01' 날짜로 시작하여 9개의 데이터 포인트를 분 단위로 생성합니다. (1분 간격)
index = pd.date_range('2000-1-1', periods=9, freq='T')  # minutely frequency

# 위에서 생성한 시간 인덱스를 사용하여, 0부터 시작하여 1씩 증가하는 숫자 시리즈를 생성합니다.
series = pd.Series(range(9), index=index)
series

2000-01-01 00:00:00    0
2000-01-01 00:01:00    1
2000-01-01 00:02:00    2
2000-01-01 00:03:00    3
2000-01-01 00:04:00    4
2000-01-01 00:05:00    5
2000-01-01 00:06:00    6
2000-01-01 00:07:00    7
2000-01-01 00:08:00    8
Freq: T, dtype: int64

주어진 시리즈를 3분 간격으로 리샘플링하고, 각 그룹에 대해 합계를 계산합니다.

In [11]:
# resamling 3T
series.resample('3T').sum()

2000-01-01 00:00:00     3
2000-01-01 00:03:00    12
2000-01-01 00:06:00    21
Freq: 3T, dtype: int64

### Upsampling

pd.date_range 함수를 사용하여 2019년 12월 31일부터 시작하는 시간 인덱스를 생성하고, 5초 간격으로 3개의 기간을 생성합니다(freq='5S'는 5-second frequency를 의미합니다).  
NumPy를 사용하여 0, 1, 3, 2, 10, 3의 값을 갖는 3x2 배열을 생성합니다.  
생성된 시간 인덱스와 배열을 사용하여 pandas 데이터 프레임을 생성하고, 열 이름으로 'col_1'과 'col_2'를 지정합니다.

In [12]:
# '2019-12-31' 날짜로 시작하여 5초 간격으로 3개의 데이터 포인트를 생성합니다. (5초 간격)
rng = pd.date_range('2019-12-31', periods=3, freq='5S')  # 5-second intervals

# 6개의 숫자를 포함하는 numpy 배열을 생성하고 이를 3x2 행렬로 재구성합니다.
# 이 행렬을 이용하여 DataFrame을 생성하며, 시간 인덱스는 rng를 사용하고, 컬럼은 'col_1', 'col_2'로 명명됩니다.
ts = pd.DataFrame(np.array([0, 1, 3, 2, 10, 3]).reshape(3, 2), 
                  index=rng, 
                  columns=['col_1', 'col_2'])
ts

,col_1,col_2
2019-12-31 00:00:00,0,1
2019-12-31 00:00:05,3,2
2019-12-31 00:00:10,10,3


In [11]:
ts_upsample = ts.resample('S').mean()
ts_upsample

,col_1,col_2
2019-12-31 00:00:00,0.0,1.0
2019-12-31 00:00:01,NaN,NaN
2019-12-31 00:00:02,NaN,NaN
2019-12-31 00:00:03,NaN,NaN
2019-12-31 00:00:04,NaN,NaN
2019-12-31 00:00:05,3.0,2.0
2019-12-31 00:00:06,NaN,NaN
2019-12-31 00:00:07,NaN,NaN
2019-12-31 00:00:08,NaN,NaN
2019-12-31 00:00:09,NaN,NaN


In [11]:
ts_upsample = ts.resample('S').mean()
ts_upsample

,col_1,col_2
2019-12-31 00:00:00,0.0,1.0
2019-12-31 00:00:01,NaN,NaN
2019-12-31 00:00:02,NaN,NaN
2019-12-31 00:00:03,NaN,NaN
2019-12-31 00:00:04,NaN,NaN
2019-12-31 00:00:05,3.0,2.0
2019-12-31 00:00:06,NaN,NaN
2019-12-31 00:00:07,NaN,NaN
2019-12-31 00:00:08,NaN,NaN
2019-12-31 00:00:09,NaN,NaN


In [15]:
# ts DataFrame을 초(second) 단위로 업샘플링 하고, 각 초마다의 평균값을 계산합니다.
# 원본 데이터가 5초 간격이기 때문에, 업샘플링 후 새로 생긴 시간 인덱스에 대해서는 값이 없어 NaN으로 표시됩니다.
ts_upsample = ts.resample('S').mean()
ts_upsample

,col_1,col_2
2019-12-31 00:00:00,0.0,1.0
2019-12-31 00:00:01,NaN,NaN
2019-12-31 00:00:02,NaN,NaN
2019-12-31 00:00:03,NaN,NaN
2019-12-31 00:00:04,NaN,NaN
2019-12-31 00:00:05,3.0,2.0
2019-12-31 00:00:06,NaN,NaN
2019-12-31 00:00:07,NaN,NaN
2019-12-31 00:00:08,NaN,NaN
2019-12-31 00:00:09,NaN,NaN


In [17]:
# 앞의 값으로 뒤의 NaN 채우기
ts_upsample.ffill()

,col_1,col_2
2019-12-31 00:00:00,0.0,1.0
2019-12-31 00:00:01,0.0,1.0
2019-12-31 00:00:02,0.0,1.0
2019-12-31 00:00:03,0.0,1.0
2019-12-31 00:00:04,0.0,1.0
2019-12-31 00:00:05,3.0,2.0
2019-12-31 00:00:06,3.0,2.0
2019-12-31 00:00:07,3.0,2.0
2019-12-31 00:00:08,3.0,2.0
2019-12-31 00:00:09,3.0,2.0


In [21]:
# 뒤의 값으로 앞의 NaN 채우기
ts_upsample.bfill()

,col_1,col_2
2019-12-31 00:00:00,0.0,1.0
2019-12-31 00:00:01,3.0,2.0
2019-12-31 00:00:02,3.0,2.0
2019-12-31 00:00:03,3.0,2.0
2019-12-31 00:00:04,3.0,2.0
2019-12-31 00:00:05,3.0,2.0
2019-12-31 00:00:06,10.0,3.0
2019-12-31 00:00:07,10.0,3.0
2019-12-31 00:00:08,10.0,3.0
2019-12-31 00:00:09,10.0,3.0


선형 보간(linear interpolation)은 두 점 사이의 값을 추정하기 위해 선형 방정식을 사용하는 방법입니다. 시간 인덱스 사이에 누락된 값이 있는 경우, 선형 보간을 사용하여 누락된 값을 채울 수 있습니다.

[선형보간법](https://ko.wikipedia.org/wiki/%EC%84%A0%ED%98%95_%EB%B3%B4%EA%B0%84%EB%B2%95)

In [23]:
# 선형보간
ts_upsample.interpolate(method='values')

,col_1,col_2
2019-12-31 00:00:00,0.0,1.0
2019-12-31 00:00:01,0.6,1.2
2019-12-31 00:00:02,1.2,1.4
2019-12-31 00:00:03,1.8,1.6
2019-12-31 00:00:04,2.4,1.8
2019-12-31 00:00:05,3.0,2.0
2019-12-31 00:00:06,4.4,2.2
2019-12-31 00:00:07,5.8,2.4
2019-12-31 00:00:08,7.2,2.6
2019-12-31 00:00:09,8.6,2.8
